In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [35]:
movies = pd.read_csv('datasets/ml-latest-small/movies.csv')
genres = movies['genres'].str.split('|', expand=True)
genres.head()

# One-hot encoding
unique_genres = genres.stack().unique()
unique_genres

genres = pd.get_dummies(genres, prefix='', prefix_sep='').groupby(level=0).sum()

movies = movies.drop('genres', axis=1).join(genres)
movies = movies.drop('title', axis=1)

movies.head()

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Mystery,Romance,Sci-Fi,Thriller,Mystery,Thriller,Thriller,Western
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
ratings = pd.read_csv('datasets/ml-latest-small/ratings.csv')
#ratings = ratings.drop('timestamp', axis=1)
ratings.head()

#generate time of day from timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

#ratings['hour'] = ratings['timestamp'].dt.hour
ratings['hour'] = ratings['timestamp'].dt.hour
ratings.drop('timestamp', axis=1, inplace=True)

#one-hot from hour column
ratings = pd.get_dummies(ratings, prefix='', prefix_sep='').groupby(level=0).sum()
ratings


,userId,movieId,rating,hour
0,1,1,4.0,18
1,1,3,4.0,18
2,1,6,4.0,18
3,1,47,5.0,19
4,1,50,5.0,18
...,...,...,...,...
100831,610,166534,4.0,21
100832,610,168248,5.0,22
100833,610,168250,5.0,19
100834,610,168252,5.0,21


In [52]:
#combine two dataframes on moviesId
df = pd.merge(movies, ratings, on='movieId')
df.head()

,movieId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Mystery,Romance,Sci-Fi,Thriller,Mystery,Thriller,Thriller,Western,userId,rating
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,4.0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,4.5
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15,2.5
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,17,4.5


In [53]:
# Create frequent itemsets using the apriori algorithm
frequent_itemsets = apriori(genres, min_support=0.03, use_colnames=True)

# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Print out the rules
print(rules)

# Print out the rules with lift over 1.2
print(rules[ (rules['lift'] >= 1.2) ])

# Print out the rules with lift over 1.2 and confidence over 0.8
print(rules[ (rules['lift'] >= 1.2) & (rules['confidence'] >= 0.8) ])

   antecedents  consequents  antecedent support  consequent support   support  \
0  (Adventure)     (Action)            0.062615            0.187641  0.062615   
1     (Action)  (Adventure)            0.187641            0.062615  0.062615   
2     (Comedy)      (Drama)            0.064053            0.042804  0.085609   
3      (Drama)     (Comedy)            0.042804            0.064053  0.085609   
4     (Comedy)    (Romance)            0.064053            0.058099  0.037569   
5    (Romance)     (Comedy)            0.058099            0.064053  0.037569   
6      (Crime)      (Drama)            0.051222            0.042804  0.040135   
7      (Drama)      (Crime)            0.042804            0.051222  0.040135   
8      (Drama)    (Romance)            0.042804            0.058099  0.036645   
9    (Romance)      (Drama)            0.058099            0.042804  0.036645   

   confidence       lift  leverage  conviction  zhangs_metric  
0    1.000000   5.329322  0.050866         i

C:\Users\erykp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [60]:
#generate time of day from timestamp in ratings


OutOfBoundsDatetime: cannot convert input with unit 'h'